In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
import suncalc

import dask.dataframe as dd
from pathlib import Path
from tqdm import tqdm
import re

import datetime as dt

import matplotlib.pyplot as plt
from matplotlib import colors
import soundfile as sf
import matplotlib.patches as patches

In [2]:
import sys

sys.path.append("../src")
sys.path.append("../src/activity")

In [3]:
import subsampling as ss
import activity_assembly as actvt
import bout.assembly as bt
import comparison.data_assembly as comp
import comparison.plot as complot
from core import SITE_NAMES

from cli import get_file_paths
import plot
import pipeline

In [4]:
avail = np.arange(0, 180, 1) + 1
reset_3 = avail[np.where((3*60 % avail) == 0)[0]]
reset_4 = avail[np.where((4*60 % avail) == 0)[0]]
reset_6 = avail[np.where((6*60 % avail) == 0)[0]]
reset_12 = avail[np.where((12*60 % avail) == 0)[0]]
reset_24 = avail[np.where((24*60 % avail) == 0)[0]]

In [5]:
reset_3

array([  1,   2,   3,   4,   5,   6,   9,  10,  12,  15,  18,  20,  30,
        36,  45,  60,  90, 180])

In [6]:
reset_4

array([  1,   2,   3,   4,   5,   6,   8,  10,  12,  15,  16,  20,  24,
        30,  40,  48,  60,  80, 120])

In [7]:
reset_6

array([  1,   2,   3,   4,   5,   6,   8,   9,  10,  12,  15,  18,  20,
        24,  30,  36,  40,  45,  60,  72,  90, 120, 180])

In [8]:
reset_12

array([  1,   2,   3,   4,   5,   6,   8,   9,  10,  12,  15,  16,  18,
        20,  24,  30,  36,  40,  45,  48,  60,  72,  80,  90, 120, 144,
       180])

In [9]:
reset_24

array([  1,   2,   3,   4,   5,   6,   8,   9,  10,  12,  15,  16,  18,
        20,  24,  30,  32,  36,  40,  45,  48,  60,  72,  80,  90,  96,
       120, 144, 160, 180])

In [10]:
f = 5
fact_ = reset_12[np.where((reset_12%f) == 0)[0]]
fact_[0] = 6
fact_

array([  6,  10,  15,  20,  30,  40,  45,  60,  80,  90, 120, 180])

In [11]:
step = 1/6
step_by = np.arange(step, 2/3+step, step)
step_by = step_by[step_by>=(1/6)]
step_by

array([0.16666667, 0.33333333, 0.5       , 0.66666667])

In [12]:
cycle_lengths = [6, 10, 12, 15, 18, 20, 24, 30, 45, 60]
percent_ons = step_by
data_params = dict()
data_params["cycle_lengths"] = cycle_lengths
data_params["percent_ons"] = percent_ons
dc_tags = ss.get_list_of_dc_tags(cycle_lengths, percent_ons)
dc_tags

['30of30',
 '1of6',
 '2of6',
 '3of6',
 '4of6',
 '2of10',
 '3of10',
 '5of10',
 '7of10',
 '2of12',
 '4of12',
 '6of12',
 '8of12',
 '2of15',
 '5of15',
 '8of15',
 '10of15',
 '3of18',
 '6of18',
 '9of18',
 '12of18',
 '3of20',
 '7of20',
 '10of20',
 '13of20',
 '4of24',
 '8of24',
 '12of24',
 '16of24',
 '5of30',
 '10of30',
 '15of30',
 '20of30',
 '8of45',
 '15of45',
 '22of45',
 '30of45',
 '10of60',
 '20of60',
 '30of60',
 '40of60']

In [13]:
site_keys = ['Carp', 'Telephone']
type_keys = ['LF', 'HF']
data_params["dc_tags"] = dc_tags
data_params['cur_dc_tag'] = '30of30'
data_params['metric_tag'] = 'bout_time_percentage'
data_params['recording_start'] = '00:00'
data_params['recording_end'] = '16:00'

for site_key in site_keys:
    for type_key in type_keys:
        print(site_key, type_key)
        data_params["site_tag"] = site_key
        data_params["site_name"] = SITE_NAMES[site_key]
        data_params["type_tag"] = type_key
        file_paths = get_file_paths(data_params)

        activity_arr, btp_arr = comp.generate_activity_btp_for_dc_schemes_and_cont(data_params, file_paths, save=True)

Carp LF


100%|██████████| 40/40 [08:18<00:00, 12.46s/it]


Carp HF


100%|██████████| 40/40 [02:22<00:00,  3.57s/it]


Telephone LF


100%|██████████| 40/40 [00:47<00:00,  1.18s/it]


Telephone HF


100%|██████████| 40/40 [08:14<00:00, 12.37s/it]


In [14]:
site_keys = ['Carp', 'Telephone']
type_keys = ['LF', 'HF']
data_params["dc_tags"] = dc_tags
data_params['cur_dc_tag'] = '30of30'
data_params['metric_tag'] = 'call_rate'
data_params['recording_start'] = '00:00'
data_params['recording_end'] = '16:00'

for site_key in site_keys:
    for type_key in type_keys:
        print(site_key, type_key)
        data_params["site_tag"] = site_key
        data_params["site_name"] = SITE_NAMES[site_key]
        data_params["type_tag"] = type_key
        file_paths = get_file_paths(data_params)

        activity_arr, callrate_arr = comp.generate_activity_call_rate_for_dc_schemes_and_cont(data_params, file_paths, save=True)

Carp LF


100%|██████████| 40/40 [02:31<00:00,  3.78s/it]


Carp HF


100%|██████████| 40/40 [00:47<00:00,  1.19s/it]


Telephone LF


100%|██████████| 40/40 [00:31<00:00,  1.28it/s]


Telephone HF


100%|██████████| 40/40 [02:40<00:00,  4.01s/it]


In [15]:
site_keys = ['Carp', 'Telephone']
type_keys = ['LF', 'HF']
data_params["dc_tags"] = dc_tags
data_params['cur_dc_tag'] = '30of30'
data_params['metric_tag'] = 'activity_index'
data_params['index_time_block_in_secs'] = 5
data_params['recording_start'] = '00:00'
data_params['recording_end'] = '16:00'

for site_key in site_keys:
    for type_key in type_keys:
        print(site_key, type_key)
        data_params["site_tag"] = site_key
        data_params["site_name"] = SITE_NAMES[site_key]
        data_params["type_tag"] = type_key
        file_paths = get_file_paths(data_params)

        activity_arr, actvtind_arr = comp.generate_activity_index_percent_for_dc_schemes_and_cont(data_params, file_paths, save=True)

Carp LF


100%|██████████| 40/40 [02:33<00:00,  3.84s/it]


Carp HF


100%|██████████| 40/40 [00:49<00:00,  1.24s/it]


Telephone LF


100%|██████████| 40/40 [00:33<00:00,  1.20it/s]


Telephone HF


100%|██████████| 40/40 [02:44<00:00,  4.10s/it]
